# Avance de regresion

Se entrena un modelo con todas las variables que no tienen valor constante para predecir la parte del valor de venta.

**NOTA: Este notebook se hizo con un computador que tenia teclado en ingles, asi que por eso podrian hacer falta las tildes**

## Librerias

In [1]:
%load_ext autoreload
%autoreload 2

import gc
import re
import seaborn as sns

import numpy as np

import pandas as pd


from sklearnex import patch_sklearn
patch_sklearn()

from typing import Tuple, List, Dict, Any
from tqdm import tqdm

from sklearn.ensemble import RandomForestRegressor

from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, mean_absolute_percentage_error

from sklearn.model_selection import train_test_split

from sklearn.feature_extraction import FeatureHasher
from sklearn.preprocessing import StandardScaler

from sklearn.model_selection import RandomizedSearchCV

Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


## Constantes 

In [2]:
# Dataset file
DATA_FILE = "data/muelle.parquet"
# Test percentage
TEST_PERC = 0.2
# Random seed
RANDOM_STATE = 42


In [3]:
# Label
LABEL_COLUMN = "Parte de valor_venta"
# Feature columns
FEATURE_COLUMNS = [
    "Destino",
    "Tipo de vuelo",
    "Sala",
    #"Muelle" Right now there is only one
    "Pasajeros saliendo",
    "AEROLÍNEA",
    "Tipo aerolínea",
    "Hora entera",
    "FECHA DIA",
    "Semana",
    "Mes",
    "Día",
    "day_of_year"
]
# Scaler
SCALER = StandardScaler()

In [4]:
PERCENTILES = [0.05, 0.25, 0.5, 0.75, 0.95]
PERCENTILE_COLS = [f"percentile__{int(perc*100)}" for perc in PERCENTILES]

## Utils

In [5]:
def hash_column(df: pd.DataFrame, col: str) -> Tuple[np.array, List[str]]:
    """Creates hashes for categorical col"""
    dimensions = int(np.log2(df[col].nunique()))
    col_hasher = FeatureHasher(
        dimensions,
        input_type="string"
    )
    hashed_values = col_hasher.fit_transform(
        df[col].values.reshape(-1, 1)
    ).todense()
    
    new_columns = [
        f"{col}_{x}" for x in range(dimensions)
    ]
    
    return hashed_values, new_columns

## Ejecucion

#### Carga de datos

In [6]:
df = pd.read_parquet(DATA_FILE)

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50775 entries, 0 to 50774
Data columns (total 18 columns):
 #   Column                       Non-Null Count  Dtype         
---  ------                       --------------  -----         
 0   Destino                      50775 non-null  object        
 1   Tipo de vuelo                50775 non-null  object        
 2   Sala                         50775 non-null  object        
 3   Muelle                       50775 non-null  object        
 4   AEROLÍNEA                    50775 non-null  object        
 5   Tipo aerolínea               50775 non-null  object        
 6   STD                          50775 non-null  datetime64[ns]
 7   Hora entera                  50775 non-null  int64         
 8   FECHA DIA                    50775 non-null  object        
 9   Pasajeros saliendo           50775 non-null  float64       
 10  Semana                       50775 non-null  int64         
 11  Mes                          50775 non-nu

In [8]:
df["day_of_year"] = df["STD"].dt.dayofyear

In [9]:
df_features = df[FEATURE_COLUMNS].copy(deep=True)

### Separacion de features numericas y categoricas

In [10]:
NUMERICAL_FEATURES = df_features.select_dtypes(include=np.number).columns
CATEGORICAL_FEATURES = df_features.select_dtypes(include="O").columns

In [11]:
NUMERICAL_FEATURES

Index(['Pasajeros saliendo', 'Hora entera', 'Semana', 'Mes', 'Día',
       'day_of_year'],
      dtype='object')

In [12]:
CATEGORICAL_FEATURES

Index(['Destino', 'Tipo de vuelo', 'Sala', 'AEROLÍNEA', 'Tipo aerolínea',
       'FECHA DIA'],
      dtype='object')

### Numero de categorias por cada feature categorica

In [13]:
for col in CATEGORICAL_FEATURES:
    print(col, df_features[col].nunique())

Destino 62
Tipo de vuelo 1
Sala 19
AEROLÍNEA 39
Tipo aerolínea 4
FECHA DIA 7


### Codificando features categoricas
Se usa onehot para features con bajo numero de categorias y hashing para el resto.

In [14]:
df_cat_features = pd.concat([
    pd.get_dummies(df["Tipo aerolínea"]),
    pd.get_dummies(df["FECHA DIA"])
],
axis=1
)

In [15]:
high_dim_dfs = []
for col in ["Destino", "Sala", "AEROLÍNEA"]:
    col_hash, col_subcols = hash_column(df, col)
    df_col_hash = pd.DataFrame(col_hash)
    df_col_hash.columns = col_subcols

    high_dim_dfs.append(df_col_hash)

df_high_dim_dfs = pd.concat(
    high_dim_dfs,
    axis=1
)

In [16]:
df_cat_features = pd.concat([
    df_cat_features,
    df_high_dim_dfs
],
axis=1)

### Anadiendo features numericas

In [17]:
df_features = pd.concat([
    df_cat_features,
    df_features[NUMERICAL_FEATURES]
], axis=1)

In [18]:
df_cat_features

,charter,legacy,low cost,regional,domingo,jueves,lunes,martes,miercoles,sabado,...,Destino_4,Sala_0,Sala_1,Sala_2,Sala_3,AEROLÍNEA_0,AEROLÍNEA_1,AEROLÍNEA_2,AEROLÍNEA_3,AEROLÍNEA_4
0,0,1,0,0,0,0,0,1,0,0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
1,0,1,0,0,0,1,0,0,0,0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
2,0,1,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
3,0,1,0,0,1,0,0,0,0,0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
4,0,1,0,0,0,0,1,0,0,0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50770,0,1,0,0,0,0,0,0,0,1,...,0.0,0.0,0.0,-1.0,0.0,-1.0,0.0,0.0,0.0,0.0
50771,0,1,0,0,1,0,0,0,0,0,...,0.0,0.0,0.0,-1.0,0.0,-1.0,0.0,0.0,0.0,0.0
50772,0,1,0,0,0,0,0,0,1,0,...,0.0,0.0,0.0,0.0,1.0,-1.0,0.0,0.0,0.0,0.0
50773,1,0,0,0,0,1,0,0,0,0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0


### Train test split

In [19]:
X_train, X_test, y_train, y_test = train_test_split(
    df_features,
    df[LABEL_COLUMN],
    test_size=TEST_PERC,
    random_state=RANDOM_STATE
)

### Escalamiento

In [20]:
X_train = SCALER.fit_transform(X_train)
X_test = SCALER.transform(X_test)

### Creacion de modelo y entrenamiento

Se usa randomized search cv para explorar hiperparametros y un randomforestregressor debido a la cantidad de columnas categoricas.

In [21]:
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

In [22]:
# Use the random grid to search for best hyperparameters
# First create the base model to tune
rf = RandomForestRegressor(n_jobs=-1)
# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
rf_random = RandomizedSearchCV(
    estimator = rf,
    param_distributions = random_grid,
    n_iter = 100,
    cv = 3,
    verbose=2,
    random_state=RANDOM_STATE,
    n_jobs = -1,
    scoring="neg_root_mean_squared_error"
)# Fit the random search model
rf_random.fit(X_train, y_train)

Fitting 3 folds for each of 100 candidates, totalling 300 fits
[CV] END bootstrap=False, max_depth=30, max_features=auto, min_samples_leaf=4, min_samples_split=2, n_estimators=2000; total time=   0.1s
[CV] END bootstrap=False, max_depth=30, max_features=sqrt, min_samples_leaf=4, min_samples_split=5, n_estimators=800; total time= 1.2min


/home/johan/miniconda3/lib/python3.10/site-packages/joblib/externals/loky/process_executor.py:700: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


[CV] END bootstrap=False, max_depth=30, max_features=auto, min_samples_leaf=4, min_samples_split=2, n_estimators=2000; total time=   0.1s
[CV] END bootstrap=False, max_depth=30, max_features=sqrt, min_samples_leaf=4, min_samples_split=5, n_estimators=800; total time= 1.3min
[CV] END bootstrap=False, max_depth=30, max_features=auto, min_samples_leaf=4, min_samples_split=2, n_estimators=2000; total time=   0.1s
[CV] END bootstrap=False, max_depth=100, max_features=sqrt, min_samples_leaf=2, min_samples_split=5, n_estimators=1000; total time= 1.8min
[CV] END bootstrap=True, max_depth=30, max_features=sqrt, min_samples_leaf=1, min_samples_split=5, n_estimators=400; total time=  44.8s
[CV] END bootstrap=False, max_depth=60, max_features=sqrt, min_samples_leaf=1, min_samples_split=5, n_estimators=600; total time= 1.1min
[CV] END bootstrap=False, max_depth=10, max_features=sqrt, min_samples_leaf=2, min_samples_split=5, n_estimators=1200; total time=  50.2s
[CV] END bootstrap=False, max_depth=6

[CV] END bootstrap=True, max_depth=None, max_features=auto, min_samples_leaf=2, min_samples_split=2, n_estimators=1800; total time=   0.0s
[CV] END bootstrap=True, max_depth=None, max_features=auto, min_samples_leaf=2, min_samples_split=2, n_estimators=1800; total time=   0.1s
[CV] END bootstrap=False, max_depth=80, max_features=sqrt, min_samples_leaf=1, min_samples_split=5, n_estimators=1400; total time= 3.1min
[CV] END bootstrap=False, max_depth=10, max_features=sqrt, min_samples_leaf=2, min_samples_split=5, n_estimators=1600; total time= 1.3min
[CV] END bootstrap=False, max_depth=70, max_features=sqrt, min_samples_leaf=1, min_samples_split=5, n_estimators=1600; total time= 3.5min
[CV] END bootstrap=True, max_depth=None, max_features=auto, min_samples_leaf=2, min_samples_split=2, n_estimators=1800; total time=   0.1s
[CV] END bootstrap=False, max_depth=80, max_features=sqrt, min_samples_leaf=1, min_samples_split=5, n_estimators=1400; total time= 3.1min
[CV] END bootstrap=False, max_d

[CV] END bootstrap=True, max_depth=50, max_features=sqrt, min_samples_leaf=4, min_samples_split=10, n_estimators=800; total time= 1.2min
[CV] END bootstrap=True, max_depth=60, max_features=sqrt, min_samples_leaf=2, min_samples_split=10, n_estimators=600; total time= 1.1min
[CV] END bootstrap=True, max_depth=90, max_features=sqrt, min_samples_leaf=4, min_samples_split=10, n_estimators=400; total time=  35.7s
[CV] END bootstrap=True, max_depth=90, max_features=auto, min_samples_leaf=2, min_samples_split=2, n_estimators=2000; total time=   0.0s
[CV] END bootstrap=True, max_depth=90, max_features=auto, min_samples_leaf=2, min_samples_split=2, n_estimators=2000; total time=   0.0s
[CV] END bootstrap=True, max_depth=90, max_features=auto, min_samples_leaf=2, min_samples_split=2, n_estimators=2000; total time=   0.0s
[CV] END bootstrap=False, max_depth=80, max_features=sqrt, min_samples_leaf=4, min_samples_split=10, n_estimators=400; total time=  34.8s
[CV] END bootstrap=False, max_depth=20, 

[CV] END bootstrap=False, max_depth=50, max_features=sqrt, min_samples_leaf=2, min_samples_split=2, n_estimators=800; total time= 1.9min
[CV] END bootstrap=True, max_depth=10, max_features=auto, min_samples_leaf=4, min_samples_split=5, n_estimators=200; total time=   0.0s
[CV] END bootstrap=True, max_depth=10, max_features=auto, min_samples_leaf=4, min_samples_split=5, n_estimators=200; total time=   0.1s
[CV] END bootstrap=True, max_depth=10, max_features=auto, min_samples_leaf=4, min_samples_split=5, n_estimators=200; total time=   0.1s
[CV] END bootstrap=True, max_depth=60, max_features=sqrt, min_samples_leaf=2, min_samples_split=10, n_estimators=600; total time= 1.1min
[CV] END bootstrap=True, max_depth=90, max_features=sqrt, min_samples_leaf=4, min_samples_split=10, n_estimators=400; total time=  35.5s
[CV] END bootstrap=False, max_depth=80, max_features=sqrt, min_samples_leaf=4, min_samples_split=10, n_estimators=400; total time=  33.0s
[CV] END bootstrap=False, max_depth=110, ma

/home/johan/miniconda3/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:425: FitFailedWarning: 
123 fits failed out of a total of 300.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
66 fits failed with the following error:
Traceback (most recent call last):
  File "/home/johan/miniconda3/lib/python3.10/site-packages/sklearn/model_selection/_validation.py", line 732, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/home/johan/miniconda3/lib/python3.10/site-packages/daal4py/sklearn/_device_offload.py", line 88, in wrapper_with_self
    return wrapper_impl(self, *args, **kwargs)
  File "/home/johan/miniconda3/lib/python3.10/site-packages/daal4py/sklearn/_device_offload.py", line 74, in wrapper

RandomizedSearchCV(cv=3, estimator=RandomForestRegressor(n_jobs=-1), n_iter=100,
                   n_jobs=-1,
                   param_distributions={'bootstrap': [True, False],
                                        'max_depth': [10, 20, 30, 40, 50, 60,
                                                      70, 80, 90, 100, 110,
                                                      None],
                                        'max_features': ['auto', 'sqrt'],
                                        'min_samples_leaf': [1, 2, 4],
                                        'min_samples_split': [2, 5, 10],
                                        'n_estimators': [200, 400, 600, 800,
                                                         1000, 1200, 1400, 1600,
                                                         1800, 2000]},
                   random_state=42, scoring='neg_root_mean_squared_error',
                   verbose=2)

### Evaluacion del modelo

In [23]:
BEST_MODEL = rf_random.best_estimator_
BEST_MODEL

RandomForestRegressor(bootstrap=False, max_depth=30, max_features='sqrt',
                      min_samples_split=10, n_estimators=1800, n_jobs=-1)

In [24]:
y_pred_train = BEST_MODEL.predict(X_train)
y_pred_test = BEST_MODEL.predict(X_test)

In [25]:
df_metrics = pd.DataFrame({
    "MAE_train": [mean_absolute_error(y_train, y_pred_train)],
    "MAE_test": [mean_absolute_error(y_test, y_pred_test)],
    "MSE_train": [mean_squared_error(y_train, y_pred_train)],
    "MSE_test": [mean_squared_error(y_test, y_pred_test)],
    "RMSE_train": [np.sqrt(mean_squared_error(y_train, y_pred_train))],
    "RMSE_test": [np.sqrt(mean_squared_error(y_test, y_pred_test))],
    "R2_train": [r2_score(y_train, y_pred_train)],
    "R2_test": [r2_score(y_test, y_pred_test)],
    "MAPE_train": [mean_absolute_percentage_error(y_train, y_pred_train)],
    "MAPE_test": [mean_absolute_percentage_error(y_test, y_pred_test)],
})

In [26]:
df_percentiles = pd.DataFrame(
    df["Parte de valor_venta"].quantile(PERCENTILES).tolist(),
    index=PERCENTILE_COLS
).T

df_metrics = pd.concat(
    [
        df_metrics,
        df_percentiles
    ],
    axis=1
)

In [27]:
df_metrics

,MAE_train,MAE_test,MSE_train,MSE_test,RMSE_train,RMSE_test,R2_train,R2_test,MAPE_train,MAPE_test,percentile__5,percentile__25,percentile__50,percentile__75,percentile__95
0,5.093986e+07,1.072202e+08,4.884917e+15,2.148279e+16,6.989218e+07,1.465701e+08,0.927113,0.672053,6.323188e+20,2.287057e+21,1.918368e+07,163861623.6,340972811.8,544858659.0,8.400272e+08


In [28]:
df_metrics.to_parquet("muelle_metrics.parquet")

In [29]:
df["Parte de valor_venta"].describe()

count    5.077500e+04
mean     3.698435e+08
std      2.583071e+08
min      0.000000e+00
25%      1.638616e+08
50%      3.409728e+08
75%      5.448587e+08
max      1.116354e+09
Name: Parte de valor_venta, dtype: float64

[CV] END bootstrap=True, max_depth=100, max_features=sqrt, min_samples_leaf=4, min_samples_split=10, n_estimators=800; total time=  43.9s
[CV] END bootstrap=True, max_depth=40, max_features=auto, min_samples_leaf=4, min_samples_split=2, n_estimators=600; total time=   0.0s
[CV] END bootstrap=True, max_depth=40, max_features=auto, min_samples_leaf=4, min_samples_split=2, n_estimators=600; total time=   0.0s
[CV] END bootstrap=True, max_depth=40, max_features=auto, min_samples_leaf=4, min_samples_split=2, n_estimators=600; total time=   0.0s
[CV] END bootstrap=False, max_depth=30, max_features=sqrt, min_samples_leaf=4, min_samples_split=2, n_estimators=600; total time=  28.8s
[CV] END bootstrap=True, max_depth=40, max_features=auto, min_samples_leaf=2, min_samples_split=10, n_estimators=2000; total time=   0.0s
[CV] END bootstrap=True, max_depth=40, max_features=auto, min_samples_leaf=2, min_samples_split=10, n_estimators=2000; total time=   0.0s
[CV] END bootstrap=True, max_depth=60, ma

## Conclusiones
- El error RMSE esta en el mismo nivel de magnitud que la mediana de los valores de Parte de valor_venta en test, mas especificamente un valor cercano a un tercio de la mediana, asi que se requiere refinamiento de datos y modelo.